# **Local Sesitive Hashing**

**Installing and importing Necessary libraies**

In [2]:
!pip install datasketch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 997 kB/s 


In [3]:

import pandas as pd
import numpy as np

import random
from random import shuffle
import re
from datasketch import MinHash, MinHashLSHForest
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive

Creating a function

In [4]:
def gen_shingles(text: str, k: int):
  shingles = []

  for i in range(len(text) - k + 1):
    shingle = text[i:i + k]
    shingles.append(shingle)

  return set(shingles)

In [5]:
df1 = 'sanjay'
df2 = 'data mining'
df3 = 'cmoe255 data mining assignmnet'
k = 3

t1 = gen_shingles(df1, k)
t2 = gen_shingles(df2, k)
t3 = gen_shingles(df3, k)

t1, t2, t3

({'anj', 'jay', 'nja', 'san'},
 {' mi', 'a m', 'ata', 'dat', 'ing', 'ini', 'min', 'nin', 'ta '},
 {' as',
  ' da',
  ' mi',
  '255',
  '5 d',
  '55 ',
  'a m',
  'ass',
  'ata',
  'cmo',
  'dat',
  'e25',
  'g a',
  'gnm',
  'ign',
  'ing',
  'ini',
  'min',
  'mne',
  'moe',
  'net',
  'ng ',
  'nin',
  'nmn',
  'oe2',
  'sig',
  'ssi',
  'ta '})

Creating a vocabulary

In [6]:
vocab = t1.union(t2).union(t3)
vocab

{' as',
 ' da',
 ' mi',
 '255',
 '5 d',
 '55 ',
 'a m',
 'anj',
 'ass',
 'ata',
 'cmo',
 'dat',
 'e25',
 'g a',
 'gnm',
 'ign',
 'ing',
 'ini',
 'jay',
 'min',
 'mne',
 'moe',
 'net',
 'ng ',
 'nin',
 'nja',
 'nmn',
 'oe2',
 'san',
 'sig',
 'ssi',
 'ta '}

encoding

In [7]:
t1_enc = [1 if s in t1 else 0 for s in vocab]
t2_enc = [1 if s in t2 else 0 for s in vocab]
t3_enc = [1 if s in t3 else 0 for s in vocab]

t1_enc

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

min hashing

In [8]:
nums = list(range(1, len(t1_enc) + 1))
print(nums)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]


In [9]:
shuffle(nums)
random_nums = nums
random_nums

[16,
 10,
 17,
 30,
 28,
 20,
 18,
 9,
 11,
 14,
 5,
 32,
 1,
 27,
 4,
 7,
 12,
 23,
 3,
 22,
 13,
 15,
 19,
 2,
 6,
 21,
 25,
 29,
 26,
 24,
 8,
 31]

In [10]:
def create_hash(size):
  hash = list(range(1, len(vocab) + 1))
  shuffle(hash)
  return hash
     

In [11]:
def build_minhash_func(vocab_size: int, nbits: int):
    hashes = []
    for i in range(nbits):
        hashes.append(create_hash(vocab_size))
    return hashes

# creating 20 minhash vectors
minhash = build_minhash_func(len(vocab), 20)


In [12]:
def create_sig(onehot_enc: list):
    # To create signatures
    signature = []
    for func in minhash:
        for i in range(1, len(vocab)+1):
            idx = func.index(i)
            if onehot_enc[idx] == 1:
                signature.append(idx)
                break
    return signature


In [13]:
t1_sig = create_sig(t1_enc)
t2_sig = create_sig(t2_enc)
t3_sig = create_sig(t3_enc)

print(t1_sig)
print(t2_sig)
     

[9, 4, 11, 11, 9, 9, 11, 4, 9, 11, 9, 11, 9, 11, 9, 9, 17, 17, 4, 9]
[0, 10, 18, 15, 18, 25, 0, 18, 25, 24, 18, 15, 24, 12, 10, 15, 31, 10, 0, 15]


finding jaccard distance

In [14]:
def jaccard(a, b):
  return len(a.intersection(b)) / len(a.union(b))

In [15]:
jaccard(t1, t2), jaccard(set(t1_sig), set(t2_sig))

(0.0, 0.0)

In [16]:
jaccard(t1, t3), jaccard(set(t1_sig), set(t3_sig))

(0.0, 0.0)

In [17]:
jaccard(t2, t3), jaccard(set(t2_sig), set(t3_sig))

(0.32142857142857145, 0.09523809523809523)

In [18]:

def fill_buckets(signatures, bands):
  n = len(signatures) // bands   
  bucket = [] 
  i = 0
  while i < len(signatures):
    bucket.append(signatures[i: i + n]) 
    i += n
  return bucket

In [19]:
band_text1 = fill_buckets(t1_sig, 10)
band_text2 = fill_buckets(t2_sig, 10)
band_text3 = fill_buckets(t3_sig, 10)

band_text1, band_text2, band_text3

([[9, 4],
  [11, 11],
  [9, 9],
  [11, 4],
  [9, 11],
  [9, 11],
  [9, 11],
  [9, 9],
  [17, 17],
  [4, 9]],
 [[0, 10],
  [18, 15],
  [18, 25],
  [0, 18],
  [25, 24],
  [18, 15],
  [24, 12],
  [10, 15],
  [31, 10],
  [0, 15]],
 [[28, 2],
  [21, 8],
  [29, 25],
  [14, 26],
  [20, 30],
  [29, 16],
  [29, 16],
  [28, 3],
  [22, 13],
  [26, 15]])

Returning candidate pairs

In [20]:
for t1, t2 in zip(band_text1, band_text2):
  if t1 == t2:
    print("Candidadte Pairs = {},{}".format(t1, t2))
    break
     

for t2, t3 in zip(band_text2, band_text3):
  if t2 == t3:
    print("Candidadte Pairs = {},{}".format(t2, t3))
    break
     

for t3, t1 in zip(band_text3, band_text1):
  if t3 == t1:
    print("Candidate Pairs = {},{}".format(t3, t1))
    break

# **Random projections**

creating 3 hyperplanes with 2 dimensions

In [21]:
nbits = 3
d = 2
     
plane_norms = np.random.rand(nbits, d) - .3
plane_norms


array([[ 0.19167126,  0.27164391],
       [ 0.31685739, -0.22675657],
       [ 0.40455319,  0.10853704]])

input

In [22]:
a = np.asarray([3, 1])
b = np.asarray([2, 1])
c = np.asarray([3, 4])
     

finding dot product

In [23]:
a_dot = np.dot(a, plane_norms.T)
b_dot = np.dot(b, plane_norms.T)
c_dot = np.dot(c, plane_norms.T)
a_dot

array([0.84665768, 0.72381558, 1.32219662])

seperating values

In [24]:
a_dot = a_dot > 0
b_dot = b_dot > 0
c_dot = c_dot > 0
a_dot

array([ True,  True,  True])

data type conversion

In [25]:
a_dot = a_dot.astype(int)
b_dot = b_dot.astype(int)
c_dot = c_dot.astype(int)
     
a_dot   

array([1, 1, 1])

In [26]:
b_dot

array([1, 1, 1])

In [27]:
c_dot

array([1, 1, 1])

convertsion and bucket creation

In [28]:
vectors = [a_dot, b_dot, c_dot]
buckets = {}
i = 0

for i in range(len(vectors)):
    hash_str = ''.join(vectors[i].astype(str))
    if hash_str not in buckets.keys():
        buckets[hash_str] = []
    buckets[hash_str].append(i)
     
print(buckets)

{'111': [0, 1, 2]}
